# Bert for summary

In [0]:
!sudo apt-get install build-essential libpoppler-cpp-dev pkg-config python-dev
# !sudo pip3 install pdftotext
!pip install transformers
!pip install xlutils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
pkg-config is already the newest version (0.29.1-0ubuntu2).
python-dev is already the newest version (2.7.15~rc1-1).
The following additional packages will be installed:
  libpoppler-cpp0v5 libpoppler-dev
The following NEW packages will be installed:
  libpoppler-cpp-dev libpoppler-cpp0v5 libpoppler-dev
0 upgraded, 3 newly installed, 0 to remove and 25 not upgraded.
Need to get 41.3 kB of archives.
After this operation, 233 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpoppler-cpp0v5 amd64 0.62.0-2ubuntu2.10 [28.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpoppler-dev amd64 0.62.0-2ubuntu2.10 [4,608 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpoppler-cpp-dev amd64 0.62.0-2ubuntu2.10 [8,684 B]
Fetched 41.3 kB in 0s

In [0]:
!cp "/content/drive/My Drive/digiledge-summary-bert-master/modeling_bertabs.py" "modeling_bertabs.py"
!cp "/content/drive/My Drive/digiledge-summary-bert-master/utils_summarization.py" "utils_summarization.py"
!cp "/content/drive/My Drive/digiledge-summary-bert-master/configuration_bertabs.py" "configuration_bertabs.py"

#### create_txt_file_year

In [0]:
import xlrd
from tqdm import tqdm

# unique1 = 'tech'
# unique1 = 'corruption'
unique1 = 'independent'
# unique2 = ''
# topics = [0,1,2,3,4,5,6,7]
file_path = '/content/drive/My Drive/dataset/Final_dataset.xlsx'
rd = xlrd.open_workbook(file_path)
table = rd.sheets()[0]
ncols = table.ncols
Years = table.col(4, start_rowx=1, end_rowx=None)
Mains = table.col(2, start_rowx=1, end_rowx=None)
main = ''
# for num in range(1,len(Topics)):
#   if topic == int(Topics[num].value):
#   main += Mains[num].value + '\n'
for i in range(len(Mains)):
  # if Years[i].value == 2005:
  if unique1 in Mains[i].value.lower():
    main += Mains[i].value.lower() + '\n'
print(main)
w = open('/content/drive/My Drive/digiledge-summary-bert-master/sample_rfp/' + unique1 + '.txt' , 'w')
w.write(main)

 we shall see more scrutiny of whether amazon, which dominates ecommerce outside china, has an unfair advantage against its own suppliers — since it both owns the network and also sells its own products on it, thus competing against them. history records that 19th century us railway companies, which owned networks and conducted commerce on them, were broken up as monopolies. already, regulators are looking into whether big tech’s forays into finance and healthcare may break existing privacy rules in those industries. but perhaps the biggest 2020 battle will be between the us and china which, despite a recent trade agreement, will continue to fight for control of the high-growth industries of the future.  each side has its warriors, whether tech companies (google, apple, amazon and facebook in the us; baidu, tencent and alibaba in china), or chipmakers (qualcomm versus huawei). one likely impact will be the continued “decoupling” of the us and chinese tech supply chains. this is support

1349574

In [0]:
#! /usr/bin/python3
import argparse
import logging
import os
import sys
import glob
from collections import namedtuple
import xlwt,xlrd
from xlutils.copy import copy

import torch
from torch.utils.data import DataLoader, SequentialSampler
from tqdm import tqdm

from modeling_bertabs import BertAbs, build_predictor
from transformers import BertTokenizer
from utils_summarization import (
    SummarizationDataset,
    build_mask,
    compute_token_type_ids,
    encode_for_summarization,
    fit_to_block_size,
)


logger = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)


Batch = namedtuple("Batch", ["document_names", "batch_size", "src", "segs", "mask_src", "tgt_str"])

def evaluate(args):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
    model = BertAbs.from_pretrained("bertabs-finetuned-cnndm")
    model.to(args.device)
    model.eval()

    symbols = {
        "BOS": tokenizer.vocab["[unused0]"],
        "EOS": tokenizer.vocab["[unused1]"],
        "PAD": tokenizer.vocab["[PAD]"],
    }

    # these (unused) arguments are defined to keep the compatibility
    # with the legacy code and will be deleted in a next iteration.
    args.result_path = ""
    args.temp_dir = ""

    data_iterator = build_data_iterator(args, tokenizer)
    predictor = build_predictor(args, tokenizer, symbols, model)

    logger.info("***** Running evaluation *****")
    logger.info("  Number examples = %d", len(data_iterator.dataset))
    logger.info("  Batch size = %d", args.batch_size)
    logger.info("")
    logger.info("***** Beam Search parameters *****")
    logger.info("  Beam size = %d", args.beam_size)
    logger.info("  Minimum length = %d", args.min_length)
    logger.info("  Maximum length = %d", args.max_length)
    logger.info("  Alpha (length penalty) = %.2f", args.alpha)
    logger.info("  Trigrams %s be blocked", ("will" if args.block_trigram else "will NOT"))

    for batch in tqdm(data_iterator):
        batch_data = predictor.translate_batch(batch)
        translations = predictor.from_batch(batch_data)
        summaries = [format_summary(t) for t in translations]
        save_summaries(summaries, summaries_output_dir, batch.document_names)


def save_summaries(summaries, path, original_document_name):
    """ Write the summaries in fies that are prefixed by the original
    files' name with the `_summary` appended.

    Attributes:
        original_document_names: List[string]
            Name of the document that was summarized.
        path: string
            Path were the summaries will be written
        summaries: List[string]
            The summaries that we produced.
    """
    # rd = xlrd.open_workbook('/content/drive/My Drive/digiledge-summary-bert-master/summaries/Bert_data.xls')
    # wt = copy(rd)
    # sh = wt.get_sheet(0)
    # for summary, document_name in zip(summaries, original_document_name):
    #     # Prepare the summary file's name
    #     if "." in document_name:
    #         bare_document_name = ".".join(document_name.split(".")[:-1])
    #         extension = document_name.split(".")[-1]
    #         name = bare_document_name
    #     else:
    #         name = document_name
    #     # file_path = os.path.join(path, name)
    #     summary = ' '.join([x.capitalize() for x in summary.split('.')])+'.'
    #     print("sum:" + summary + "\tdoc:" + name)
    #     sh.write(int(name),4,summary)
    for summary, document_name in zip(summaries, original_document_name):
        # Prepare the summary file's name
        if "." in document_name:
            bare_document_name = ".".join(document_name.split(".")[:-1])
            extension = document_name.split(".")[-1]
            name = bare_document_name + "_summary." + extension
        else:
            name = document_name + "_summary"
        file_path = os.path.join(path, name)
        # summary = ' '.join([x.capitalize() for x in summary.split('.')])+'.'
        with open(file_path, "w") as output:
            output.write(summary)

    # sum_docs = zip(summaries, original_document_name)
    # for i in sum_docs:
    #   summrization = i[0]
    #   docnum = i[1].split('.')[0]
    #   rd = xlrd.open_workbook('/content/drive/My Drive/digiledge-summary-bert-master/summaries/Bert_data.xls')
    #   wt = copy(rd)
    #   sh = wt.get_sheet(0)
    #   sh.write(int(docnum),4,summrization)
    #     with open(file_path, "w") as output:
    #         output.write(summary)  
    
    # wt.save('/content/drive/My Drive/digiledge-summary-bert-master/summaries/Bert_data.xls')

    # list = os.listdir('/content/drive/My Drive/digiledge-summary-bert-master/sample_rfp/')
    # for i in list:
    #   os.remove('/content/drive/My Drive/digiledge-summary-bert-master/sample_rfp/' + i)


def format_summary(translation):
    """ Transforms the output of the `from_batch` function
    into nicely formatted summaries.
    """
    raw_summary, _, _ = translation
    summary = (
        raw_summary.replace("[unused0]", "")
        .replace("[unused3]", "")
        .replace("[PAD]", "")
        .replace("[unused1]", "")
        .replace(r" +", " ")
        .replace(" [unused2] ", ". ")
        .replace("[unused2]", "")
        .strip()
    )

    summary_lines = summary.split('.')
    summary_lines = [line.strip().capitalize() for line in summary_lines]
    summary_lines = [line+'.' for line in summary_lines]
    summary = ' '.join(summary_lines)

    return summary


#
# LOAD the dataset
#


def build_data_iterator(args, tokenizer):
    dataset = load_and_cache_examples(args, tokenizer)
    sampler = SequentialSampler(dataset)

    def collate_fn(data):
        return collate(data, tokenizer, block_size=512, device=args.device)

    iterator = DataLoader(dataset, sampler=sampler, batch_size=args.batch_size, collate_fn=collate_fn,)

    return iterator


def load_and_cache_examples(args, tokenizer):
    dataset = SummarizationDataset(documents_dir)
    print("dataset type : " + str(type(dataset)))
    return dataset


def collate(data, tokenizer, block_size, device):
    """ Collate formats the data passed to the data loader.

    In particular we tokenize the data batch after batch to avoid keeping them
    all in memory. We output the data as a namedtuple to fit the original BertAbs's
    API.
    """
    data = [x for x in data if not len(x[1]) == 0]  # remove empty_files
    names = [name for name, _, _ in data]
    summaries = [" ".join(summary_list) for _, _, summary_list in data]

    encoded_text = [encode_for_summarization(story, summary, tokenizer) for _, story, summary in data]
    encoded_stories = torch.tensor(
        [fit_to_block_size(story, block_size, tokenizer.pad_token_id) for story, _ in encoded_text]
    )
    encoder_token_type_ids = compute_token_type_ids(encoded_stories, tokenizer.cls_token_id)
    encoder_mask = build_mask(encoded_stories, tokenizer.pad_token_id)

    batch = Batch(
        document_names=names,
        batch_size=len(encoded_stories),
        src=encoded_stories.to(device),
        segs=encoder_token_type_ids.to(device),
        mask_src=encoder_mask.to(device),
        tgt_str=summaries,
    )

    return batch


def decode_summary(summary_tokens, tokenizer):
    """ Decode the summary and return it in a format
    suitable for evaluation.
    """
    summary_tokens = summary_tokens.to("cpu").numpy()
    summary = tokenizer.decode(summary_tokens)
    sentences = summary.split(".")
    sentences = [s + "." for s in sentences]
    return sentences


def main():
    """ The main function defines the interface with the users.
    """
    global documents_dir
    global summaries_output_dir
    documents_dir = "/content/drive/My Drive/digiledge-summary-bert-master/sample_rfp"
    summaries_output_dir = "/content/drive/My Drive/digiledge-summary-bert-master/summaries"

    parser = argparse.ArgumentParser()
    # parser.add_argument("-documents_dir",default="/content/drive/My Drive/digiledge-summary-bert-master/sample_rfp/",type=str,required=True,help="/content/drive/My Drive/digiledge-summary-bert-master/sample_rfp",)
    # parser.add_argument("-summaries_output_dir",default="/content/drive/My Drive/digiledge-summary-bert-master/summaries",type=str,required=True,help="/content/drive/My Drive/digiledge-summary-bert-master/summaries",)

    # EVALUATION options
    parser.add_argument(
        "-no_cuda", default=False, type=bool, help="Whether to force the execution on CPU.",
    )
    parser.add_argument(
        "-batch_size", default=4, type=int, help="Batch size per GPU/CPU for training.",
    )
    # BEAM SEARCH arguments
    parser.add_argument(
        "-min_length", default=100, type=int, help="Minimum number of tsokens for the summaries.",
    )
    parser.add_argument(
        "-max_length", default=250, type=int, help="Maixmum number of tokens for the summaries.",
    )
    parser.add_argument(
        "-beam_size", default=10, type=int, help="The number of beams to start with for each example.",
    )
    parser.add_argument(
        "-alpha", default=0.95, type=float, help="The value of alpha for the length penalty in the beam search.",
    )
    parser.add_argument(
        "-block_trigram",
        default=True,
        type=bool,
        help="Whether to block the existence of repeating trigrams in the text generated by beam search.",
    )
    
    args, unknown = parser.parse_known_args()

    # Select device (distibuted not available)
    args.device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")

    # Check the existence of directories
    if not summaries_output_dir:
        summaries_output_dir = documents_dir

    if not documents_dir_is_valid(documents_dir):
        raise FileNotFoundError(
            "We could not find the directory you specified for the documents to summarize, or it was empty. Please specify a valid path."
        )
    os.makedirs(summaries_output_dir, exist_ok=True)
    
    # convert_pdf_to_text(documents_dir, documents_dir)

    evaluate(args)


# def convert_pdf_to_text(source_path, destination_path):
#     """ Write the extracted text from pdf documents in
#     files that are prefixed by the original documents

#     Attributes:
#         source_path: string
#             Path were the PDF documents are placed
#         destination_path: string
#             Path were the extracted texts are placed
#     """

#     for doc in glob.glob(source_path+'*.pdf'):
#         with open(doc, 'rb') as f_p:
#             text = pdftotext.PDF(f_p)
#         logger.info('Read {} pages from document {}'.format(len(text), doc))
#         doc_text = '\n\n'.join(text)
#         doc_name = doc.split('/')[-1].replace('pdf', 'txt')
#         with open(destination_path+doc_name, 'w') as f_t:
#             f_t.write(doc_text)

def documents_dir_is_valid(path):
    if not os.path.exists(path): return False
    file_list = os.listdir(path)
    if len(file_list) == 0: return False
    return True


if __name__ == "__main__":
    main()


INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/remi/bertabs-finetuned-cnndm-extractive-abstractive-summarization-config.json from cache at /root/.cache/torch/transformers/7ebb4ac81007d10b400cb6c2968d4c8f1275a3e0cc3bab7f20f81913198b542c.c3ca02f3f42c25433be972663dafb463df23976fd811cdae89d87c8c4d74ab40
INFO:transformers.configuration_utils:Model config BertAbsConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "bos_token_id": null,
  "dec_dropout": 0.2,
  "dec_ff_size": 2048,
  "dec_heads": 8,
  "dec_hidden_size": 768,
  "dec_layers": 6,
  "do_sample": false,
  "enc_dropout": 0.2,
  "enc_ff_size": 512,
  "enc_hea


100%|██████████| 2/2 [00:43<00:00, 27.35s/it]


#### Remove txt files

In [0]:
import os
list = os.listdir('/content/drive/My Drive/digiledge-summary-bert-master/sample_rfp/')
for i in list:
  os.remove('/content/drive/My Drive/digiledge-summary-bert-master/sample_rfp/' + i)

#### Remove summary files

In [0]:
list = os.listdir('/content/drive/My Drive/digiledge-summary-bert-master/summaries/')
for i in list:
  os.remove('/content/drive/My Drive/digiledge-summary-bert-master/summaries/' + i)